# Cafe Sale Analysis

#### Life cycle of Sales Dataset Project

- Understanding the Problem Statement
- Data Checks to perform
- Exploratory data analysis
- Data Pre-Processing
- Model Training
- Choose best model

### Problem Statement

-   The cafe sales dataset contains missing, erroneous, and unknown values across multiple columns, requiring comprehensive data cleaning to replace or remove inconsistent entries and ensure data integrity for accurate analysis.

### Import Data and required packages

importing SQLconnector, Pandas, Numpy, Matplotlib and Warnings Liabrary

In [1]:
import mysql.connector 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

#### Connect to the MySQL Database

In [2]:
connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password="1406",
    database="cafe_sale"
)
cursor = connection.cursor()

Run the query

In [3]:
query = "select * from dirty_cafe_sales"

Connection close

In [4]:
dff = pd.read_sql(query,connection)
connection.close()

Copy the data set

In [5]:
df = dff.copy()

## General Information of the dataset

Showing some top rows

In [6]:
df.head(10)

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,ERROR,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11
5,TXN_2602893,Smoothie,5,4.0,20.0,Credit Card,,2023-03-31
6,TXN_4433211,UNKNOWN,3,3.0,9.0,ERROR,Takeaway,2023-10-06
7,TXN_6699534,Sandwich,4,4.0,16.0,Cash,UNKNOWN,2023-10-28
8,TXN_4717867,,5,3.0,15.0,,Takeaway,2023-07-28
9,TXN_2064365,Sandwich,5,4.0,20.0,,In-store,2023-12-31


info of nummerical columns

In [7]:
df.describe()

,Quantity,Price Per Unit
count,9006.000000,9006.000000
mean,3.023540,2.951865
std,1.418644,1.277659
min,1.000000,1.000000
25%,2.000000,2.000000
50%,3.000000,3.000000
75%,4.000000,4.000000
max,5.000000,5.000000


Shape of the dataset

In [8]:
print(df.shape)

(9006, 8)


showing all columns of the dataset

In [9]:
for i in df.columns:
    print(i)

Transaction ID
Item
Quantity
Price Per Unit
Total Spent
Payment Method
Location
Transaction Date


Check data types

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9006 entries, 0 to 9005
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Transaction ID    9006 non-null   object 
 1   Item              9006 non-null   object 
 2   Quantity          9006 non-null   int64  
 3   Price Per Unit    9006 non-null   float64
 4   Total Spent       9006 non-null   object 
 5   Payment Method    9006 non-null   object 
 6   Location          9006 non-null   object 
 7   Transaction Date  9006 non-null   object 
dtypes: float64(1), int64(1), object(6)
memory usage: 563.0+ KB


Showing Unique Values to each Columns

In [11]:
df.nunique()

Transaction ID      9006
Item                  11
Quantity               5
Price Per Unit         6
Total Spent           20
Payment Method         6
Location               5
Transaction Date     368
dtype: int64

Check the Missing Valueas

In [12]:
df.isnull().sum()

Transaction ID      0
Item                0
Quantity            0
Price Per Unit      0
Total Spent         0
Payment Method      0
Location            0
Transaction Date    0
dtype: int64

### Data Pre-processing and Cleanning

In [8]:
df.replace("null", np.nan,regex=True, inplace=True)

In [9]:
print(df.isnull().sum())

Transaction ID      0
Item                0
Quantity            0
Price Per Unit      0
Total Spent         0
Payment Method      0
Location            0
Transaction Date    0
dtype: int64


In [10]:
df['Item'].isnull().sum()

np.int64(0)

In [12]:
print(df['Transaction ID'].unique())
print(df['Location'].unique())
print("categories in 'Item' variable:  ",end="")
print(df['Item'].unique())

['TXN_1961373' 'TXN_4977031' 'TXN_4271903' ... 'TXN_7672686' 'TXN_5255387'
 'TXN_6170729']
['Takeaway' 'In-store' 'UNKNOWN' '' 'ERROR']
categories in 'Item' variable:  ['Coffee' 'Cake' 'Cookie' 'Salad' 'Smoothie' 'UNKNOWN' 'Sandwich' ''
 'ERROR' 'Juice' 'Tea']


In [13]:
# print((df['Location']=="").sum())
# print((df['Location']=="ERROR").sum())
# print((df['Location']=="UNKNOWN").sum())
print((df['Location'].isin(['','ERROR','UNKNOWN']).sum()))

3564


In [14]:
df['Location'].replace(["","ERROR","UNKNOWN"], "UNKNOWN", inplace=True)

In [15]:
df['Location'].unique()

array(['Takeaway', 'In-store', 'UNKNOWN'], dtype=object)

In [16]:
print((df['Item'].isin(['','ERROR','UNKNOWN']).sum()))

859


In [17]:
df['Item'].replace(["", "ERROR", "UNKNOWN"],"Coffee", inplace=True)
print(df['Item'].unique())

['Coffee' 'Cake' 'Cookie' 'Salad' 'Smoothie' 'Sandwich' 'Juice' 'Tea']


In [18]:
print(df['Quantity'].unique())
print(df['Price Per Unit'].unique())
print(df['Total Spent'].isin(["", "UNKNOWN","ERROR"]).sum())

[2 4 5 3 1]
[2.  3.  1.  5.  4.  1.5]
462


- Replace these fake missing values with np.nan

In [19]:
df['Total Spent'] = df['Total Spent'].replace(["", "ERROR", "UNKNOWN"], np.nan)

In [20]:
mask_1 = df['Total Spent'].isnull() & df['Quantity'].notnull() & df['Price Per Unit'].notnull()
df.loc[mask_1, 'Total Spent'] = df.loc[mask_1, 'Quantity'] * df.loc[mask_1, 'Price Per Unit']

In [21]:
df['Quantity'] = df['Quantity'].replace(["", "ERROR", "UNKNOWN"],np.nan)

In [22]:
mask_2 = df['Quantity'].isnull() & df['Price Per Unit'].notnull() & df['Total Spent'].notnull()
df.loc[mask_2, 'Quantity'] = df.loc[mask_2, 'Total Spent'] / df.loc[mask_2, 'Price Per Unit']

In [23]:
df['Price Per Unit'] = df['Price Per Unit'].replace(["", "ERROR", "UNKNOWN"],np.nan)

In [24]:
mask_3 = df['Price Per Unit'].isnull() & df['Quantity'].notnull() & df['Total Spent'].notnull()
df.loc[mask_3, 'Price Per Unit'] = df.loc[mask_3, 'Total Spent'] / df.loc[mask_3, 'Quantity']

In [26]:
print(df['Quantity'].unique())
print(df['Price Per Unit'].unique())
print(df['Total Spent'].isin(["", "UNKNOWN","ERROR"]).sum())

[2 4 5 3 1]
[2.0 3.0 1.0 5.0 4.0 1.5]
0


In [27]:
df.isnull().sum()

Transaction ID      0
Item                0
Quantity            0
Price Per Unit      0
Total Spent         0
Payment Method      0
Location            0
Transaction Date    0
dtype: int64

In [28]:
df['Transaction Date'] = df['Transaction Date'].replace(["", "ERROR", "UNKNOWN"],np.nan)

- Only remove the null values

In [29]:
df = df.dropna(subset=['Transaction Date'])

In [30]:
df['Transaction Date'].isnull().sum()

np.int64(0)

In [32]:
# print(df['Transaction ID'].unique())
# print(df['Item'].unique())
# print(df['Quantity'].unique())
# print(df['Price Per Unit'].unique())
# print(df['Total Spent'].unique())
# print(df['Payment Method'].unique())
# print(df['Location'].unique())
# print(df['Transaction Date'].unique())

for col in df.columns:
    print(f"{col}: {df[col].unique()}")

Transaction ID: ['TXN_1961373' 'TXN_4977031' 'TXN_4271903' ... 'TXN_7672686' 'TXN_5255387'
 'TXN_6170729']
Item: ['Coffee' 'Cake' 'Cookie' 'Salad' 'Smoothie' 'Sandwich' 'Juice' 'Tea']
Quantity: [2 4 5 3 1]
Price Per Unit: [2.0 3.0 1.0 5.0 4.0 1.5]
Total Spent: ['4.0' '12.0' 4.0 '10.0' '20.0' '9.0' '16.0' '15.0' '25.0' '8.0' '5.0'
 '3.0' '6.0' 12.0 2.0 3.0 '2.0' '1.0' '7.5' 9.0 '1.5' 6.0 '4.5' 20.0 7.5
 15.0 10.0 25.0 8.0 1.0 16.0 1.5 4.5 5.0]
Payment Method: ['Credit Card' 'Cash' 'UNKNOWN' 'Digital Wallet' 'ERROR' '']
Location: ['Takeaway' 'In-store' 'UNKNOWN']
Transaction Date: ['2023-09-08' '2023-05-16' '2023-07-19' '2023-04-27' '2023-06-11'
 '2023-03-31' '2023-10-06' '2023-10-28' '2023-07-28' '2023-12-31'
 '2023-11-07' '2023-05-03' '2023-06-01' '2023-03-21' '2023-11-15'
 '2023-06-10' '2023-02-24' '2023-03-25' '2023-01-15' '2023-03-30'
 '2023-12-01' '2023-09-18' '2023-06-03' '2023-12-13' '2023-04-20'
 '2023-04-10' '2023-03-11' '2023-06-02' '2023-11-06' '2023-08-15'
 '2023-10-09' '202

In [33]:
print(df['Item'].value_counts())

Item
Coffee      1835
Juice       1020
Cake         989
Salad        985
Sandwich     969
Smoothie     951
Cookie       929
Tea          918
Name: count, dtype: int64


In [34]:
print(df['Payment Method'].value_counts())

Payment Method
                  2188
Digital Wallet    1980
Cash              1957
Credit Card       1955
ERROR              261
UNKNOWN            255
Name: count, dtype: int64


In [35]:
df['Payment Method'] = df['Payment Method'].replace(['', 'ERROR', 'UNKNOWN'], 'UNKNOWN')

In [36]:
print(df['Payment Method'].value_counts(normalize=True)*100)

Payment Method
UNKNOWN           31.456491
Digital Wallet    23.033969
Cash              22.766403
Credit Card       22.743136
Name: proportion, dtype: float64


In [37]:
print(df['Transaction ID'].unique())
print(df['Item'].unique())
print(df['Quantity'].unique())
print(df['Price Per Unit'].unique())
print(df['Total Spent'].unique())
print(df['Payment Method'].unique())
print(df['Location'].unique())

['TXN_1961373' 'TXN_4977031' 'TXN_4271903' ... 'TXN_7672686' 'TXN_5255387'
 'TXN_6170729']
['Coffee' 'Cake' 'Cookie' 'Salad' 'Smoothie' 'Sandwich' 'Juice' 'Tea']
[2 4 5 3 1]
[2.0 3.0 1.0 5.0 4.0 1.5]
['4.0' '12.0' 4.0 '10.0' '20.0' '9.0' '16.0' '15.0' '25.0' '8.0' '5.0'
 '3.0' '6.0' 12.0 2.0 3.0 '2.0' '1.0' '7.5' 9.0 '1.5' 6.0 '4.5' 20.0 7.5
 15.0 10.0 25.0 8.0 1.0 16.0 1.5 4.5 5.0]
['Credit Card' 'Cash' 'UNKNOWN' 'Digital Wallet']
['Takeaway' 'In-store' 'UNKNOWN']


In [38]:
mask_4 = (df['Payment Method'].str.lower() == 'unknown') & (df['Item'].str.lower() == 'unknwon Item') & (df['Location'].str.lower() == 'unknown')
df = df[~mask_4]

### create one clean CSV file

In [114]:
df.to_csv('Clean Cafe sales_final.csv', index=False)